In [2]:
!pip install streamlit PyPDF2 sentence-transformers faiss-cpu openai
import streamlit as st
import PyPDF2
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import openai

# Set up OpenAI API key
openai.api_key = "your_openai_api_key"

# Load Sentence Transformer model for embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')

# Initialize FAISS index
dimension = 384  # Dimension of the embeddings
index = faiss.IndexFlatL2(dimension)

# Streamlit app
st.title("AI Research Paper Q&A")
st.write("Upload an AI research paper in PDF format and ask questions!")

# File upload
uploaded_file = st.file_uploader("Upload a PDF file", type="pdf")

if uploaded_file is not None:
    # Extract text from PDF
    pdf_reader = PyPDF2.PdfReader(uploaded_file)
    text = ""
    for page in pdf_reader.pages:
        text += page.extract_text()

    # Preprocess text into chunks
    chunks = [text[i:i + 500] for i in range(0, len(text), 500)]  # Split into 500-character chunks

    # Generate embeddings for chunks
    embeddings = model.encode(chunks)

    # Add embeddings to FAISS index
    index.add(np.array(embeddings))

    # User query
    query = st.text_input("Ask a question about the paper:")
    if query:
        # Embed the query
        query_embedding = model.encode([query])

        # Search for the most relevant chunk
        distances, indices = index.search(query_embedding, k=1)  # Retrieve top 1 relevant chunk
        relevant_chunk = chunks[indices[0][0]]

        # Generate response using OpenAI GPT
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are a helpful assistant that answers questions about research papers."},
                {"role": "user", "content": f"Context: {relevant_chunk}\n\nQuestion: {query}\n\nAnswer:"}
            ]
        )

        # Display the response
        st.write("**Answer:**")
        st.write(response['choices'][0]['message']['content'])

        # Display the relevant chunk for reference
        st.write("**Relevant Section from the Paper:**")
        st.write(relevant_chunk)

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 5.1.5 requires pyqt5<5.13, which is not installed.
spyder 5.1.5 requires pyqtwebengine<5.13, which is not installed.
daal4py 2021.3.0 requires daal==2021.2.3, which is not installed.
torchvision 0.19.0 requires numpy<2, but you have numpy 2.0.2 which is incompatible.
tensorflow-intel 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.0.2 which is incompatible.
tensorflow-intel 2.12.0 requires protobuf!=4.21.0,!=4.21.1,!=4.21.2,!=4.21.3,!=4.21.4,!=4.21.5,<5.0.0dev,>=3.20.3, but you have protobuf 3.20.1 which is incompatible.
numba 0.54.1 requires numpy<1.21,>=1.17, but you have numpy 2.0.2 which is incompatible.


  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
  Attempting uninstall: scipy
    Found existing installation: scipy 1.7.1
    Uninstalling scipy-1.7.1:
      Successfully uninstalled scipy-1.7.1
  Attempting uninstall: pygments
    Found existing installation: Pygments 2.10.0
    Uninstalling Pygments-2.10.0:
      Successfully uninstalled Pygments-2.10.0
  Attempting uninstall: anyio
    Found existing installation: anyio 2.2.0
    Uninstalling anyio-2.2.0:
      Successfully uninstalled anyio-2.2.0


ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject